In [1]:
#Import required packages.

import urllib  
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
#Setting up parser preliminiaries.

opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent', 'Google Chrome/57.0.2987.133')]

url=('http://www.trademe.co.nz')
ourURL=opener.open(url).read()

In [3]:
def urlmaker(page):
    return('http://www.trademe.co.nz/browse/categorylistings.aspx?rptpath=344-422-&mcatpath=mobile-phones%2fmobile-phones&page='
           +str(page))
urlmaker(0)

#Simple function which returns page number for mobile phone listings, ie: ulrmaker(5)= ulr for page 5 of listings.

'http://www.trademe.co.nz/browse/categorylistings.aspx?rptpath=344-422-&mcatpath=mobile-phones%2fmobile-phones&page=0'

In [4]:
def soupify(page):
    return(BeautifulSoup(opener.open(urlmaker(page)).read(),"lxml") )
#soupify(0)

#Returns .html file for a given page.

In [5]:
#this is every instance of soupify for every page. Doing this way to reduce function clutter+ save a LOT of time

soups=[]

for i in range(0,120):
        soups.append(soupify(i))

#soups[0]

In [9]:
#This function trims the .html into a single string for each listing. The output is a list: item(page)[listing]

def item(page):
    p=soups[page].findAll('a', {'class':re.compile("tile")})
    paragraphs=[]
    
    for x in p:
        paragraphs.append(str(x))
    for i in range(len(paragraphs)):  
        paragraphs[i]=" ".join(paragraphs[i].split())
    return(paragraphs)

#item(0)[1]

In [12]:
#Using the same logic as soups: iterate this over each item in soups: so we are not reliant on the item() function.
items=[]
for j in range(len(soups)):
        items.append(item(j))
     

In [13]:
#here logic is applied to listing strings: defining search functions.

def location(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="location">(.+?)</div>', 
                               items[page][i]).group(1)) 
    return(liste) 

def name(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="title">(.+?)</div>', 
                               items[page][i]).group(1))  
    return(liste)

def bidprices(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('<div class="listingBidPrice" id="SuperGridGallery_ctl00_BidInfo_listingBidPrice">(.+?)</div>'
                                   , items[page][i]).group(1))    
        except:
            liste.append('BuyNowOnly')   
    return(liste)

def buynowprices(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('<div class="listingBuyNowPrice" id="SuperGridGallery_ctl00_BuyNow_listingBuyNowPrice">(.+?)</div>'
                                   , items[page][i]).group(1))    
        except:
            liste.append('BidOnly')
    return(liste)


def buynowalt(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('<div class="listingBuyNowPrice with-was-price" id="SuperGridGallery_ctl00_BuyNow_listingBuyNowPrice">(.+?)</div>'
                                   , items[page][i]).group(1))    
        except:
            liste.append('NaN')
    return(liste)

def buynowalt2(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('div class="listingBuyNowPrice with-was-price with-was-price with-was-price" id="SuperGridGallery_ctl00_BuyNow_listingBuyNowPrice">(.+?)</div>'
                                   , items[page][i]).group(1))    
        except:
            liste.append('NaN')
    return(liste)

def ID(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="watchlist " id="watchlist(.+?)onclick', 
                               items[page][i]).group(1))  
    return(liste)


'Enter logic either side of (.+?) {in the red}'

'Enter logic either side of (.+?) {in the red}'

In [12]:
#again: applying this list logic. Some inconsistency in how prices are listed calls for the different bid/buynow lists.

locations=[]

for i in range(len(soups)):
    locations.append(location(i))
    
flattened_locations = [val for sublist in locations for val in sublist]
#flattened_locations

names=[]

for i in range(len(soups)):
    names.append(name(i))
    
flattened_names = [val for sublist in names for val in sublist]
#flattened_names

bidpricess=[]

for i in range(len(soups)):
    bidpricess.append(bidprices(i))
    
flattened_bidprices = [val for sublist in bidpricess for val in sublist]
#flattened_bidprices

buynowpricess=[]

for i in range(len(soups)):
    buynowpricess.append(buynowprices(i))
    
flattened_buynowprices = [val for sublist in buynowpricess for val in sublist]
#flattened_buynowpricess

buynowpricessalt=[]

for i in range(len(soups)):
    buynowpricessalt.append(buynowalt(i))
    
flattened_buynowpricesalt = [val for sublist in buynowpricessalt for val in sublist]
#flattened_buynowpricess

buynowpricessalt2=[]

for i in range(len(soups)):
    buynowpricessalt2.append(buynowalt2(i))
    
flattened_buynowpricesalt2 = [val for sublist in buynowpricessalt2 for val in sublist]
#flattened_buynowpricess

IDS=[]

for i in range(len(soups)):
    IDS.append(ID(i))
    
flattened_IDS = [val for sublist in IDS for val in sublist]
#flattened_IDS

In [13]:
#Create a dataframe to append to.

Frame=pd.DataFrame()

Frame['Location']=flattened_locations
Frame['Item_Name']=flattened_names
Frame['BidPrice']=flattened_bidprices
Frame['BuyNow_Price']=flattened_buynowprices
Frame['Price_Fixer']=flattened_buynowpricesalt
Frame['Price_Fixer2']=flattened_buynowpricesalt2
Frame['ID']=flattened_IDS

In [14]:
#Time to filtered

#To do: do this froma list, like below. Very unasthetic at the moment.
filtered=Frame[Frame.Item_Name.str.contains("Screen") == False]
filtered=filtered[filtered.Item_Name.str.contains("screen") == False]
filtered=filtered[filtered.Item_Name.str.contains("Case") == False]
filtered=filtered[filtered.Item_Name.str.contains("case") == False]
filtered=filtered[filtered.Item_Name.str.contains("Lense") == False]
filtered=filtered[filtered.Item_Name.str.contains("lense") == False]
filtered=filtered[filtered.Item_Name.str.contains("Trade") == False]
filtered=filtered[filtered.Item_Name.str.contains("trade") == False]
filtered=filtered[filtered.Item_Name.str.contains("adapter") == False]
filtered=filtered[filtered.Item_Name.str.contains("Adapter") == False]
filtered=filtered[filtered.Item_Name.str.contains("Dock") == False]
filtered=filtered[filtered.Item_Name.str.contains("dock") == False]
filtered=filtered[filtered.Item_Name.str.contains("parts") == False]
filtered=filtered[filtered.Item_Name.str.contains("Parts") == False]

print(len(filtered))

#List to use if doable in a one liner

#to_remove=["Screen","screen","Case","case","Lense","lense","Trade","trade",
          #"adapter","Adapter","Dock","dock","parts","Parts"]

#filtered

5912


In [15]:
#This is sorting the different prices into correct places.

lister=[]

for i in range(len(filtered)):
    if filtered.iloc[i]['BidPrice']=='BuyNowOnly' and filtered.iloc[i]['BuyNow_Price']=='BidOnly':
        lister.append(filtered.iloc[i]['Price_Fixer'])
    else:
         lister.append(filtered.iloc[i]['BuyNow_Price'])
#lister   

In [16]:
#Furthering this sort. This isn't very pythonic.

filtered['BuyNowPrice']=lister
filtered

lister2=[]

for i in range(len(filtered)):
    if filtered.iloc[i]['BuyNowPrice']=='NaN':
        lister2.append(filtered.iloc[i]['Price_Fixer2'])
    else:
         lister2.append(filtered.iloc[i]['BuyNowPrice'])
            
filtered['BuyNowPricee']=lister2
#lister   

del filtered['BuyNow_Price']
del filtered['Price_Fixer']
del filtered['BuyNowPrice']
del filtered['Price_Fixer2']

In [17]:
#For naming the .csv

import time
timestr = time.strftime("%Y+%m+%d-%H+%M+%S")
timestr

'2017+06+02-09+14+55'

In [20]:
#finalizing. Deleting exepction cases, duplicate rows.

filtered=filtered[filtered.BuyNowPricee.str.contains("NaN") == False]

path='D:\\TradeMe2\\'

df_obj = filtered2.select_dtypes(['object'])
filtered2[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

filtered2.to_csv(path+'filtered_data'+timestr+'.csv', dwdelimiter=',', encoding='utf-8')